### Importation des librairies

In [55]:
from dicoo import tickers_brut
from creation_db import *
from import_data import *
from strategies import *
from model import *
from base_update import *
from metrics import *
import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Création de la database

In [56]:
create_tables()
create_initial_portfolios()
generate_clients()
generate_managers()
generate_products()

Tables créées avec succès.
3 portefeuilles de base créés avec succès.
10 clients générés avec un profil de risque assigné aléatoirement.
5 managers générés avec succès.
Produits ajoutés avec succès.


### Importation des données

In [57]:
# Importation des données historiques et remplissage de la table returns 
# (nous avons besoin de minimum 252 observations pour les stratégies choissies)
data_importer = DataImporter(db_file="fund.db")
data_importer.fill_returns(start_date = "2019-01-01",end_date = "2022-12-25")

2025-03-30 22:48:37,349 - ERROR - 
1 Failed download:
2025-03-30 22:48:37,350 - ERROR - ['NG=F']: YFInvalidPeriodError("NG=F: Period 'max' is invalid, must be of the format 1d, 5d, etc.")


Returns hebdomadaires ajoutés avec succès: 102897 entrées.


### Entraînement du model pour la strategie "low_turnover"

In [52]:
fit_model(start_date = "2019-01-01", end_date = "2022-12-31")

Le modèle a été entraîné et sauvegardé dans model.pkl


LinearRegression()

### Lancement hebdomadaire (tous les lundis) du code afin de calculer les nouveaux portefeuilles et de mettre à jour la base de données (tables : returns, portefeuilles, deals)

In [53]:
strategies = Strategies(db_file="fund.db")

In [54]:
nb_deals = 0
prev_month = None

# Boucle sur chaque semaine (ici on choisit les lundis comme jours de traitement)
for current_date in pd.date_range(start="2023-01-02", end="2024-12-12", freq="W-MON"):
    date_str = current_date.strftime("%Y-%m-%d")
    
    # Détection d'un nouveau mois : réinitialisation du compteur de deals
    month_year = current_date.strftime("%Y-%m")
    if month_year != prev_month:
        nb_deals = 0
        prev_month = month_year

    # 1. Insertion des returns de la semaine précédente dans la base de données
    week_start = (current_date - timedelta(days=7)).strftime("%Y-%m-%d")
    week_end   = date_str
    print(f"Téléchargement des returns entre {week_start} et {week_end}")
    data_importer.fill_returns(week_start, week_end)

    # 2. Calcul des portefeuilles optimaux pour chaque stratégie
    df_low_risk  = strategies.low_risk() 
    df_low_turnover = strategies.linear_strategy(date_str) 
    df_high_yield = strategies.high_yield() 

    # 3. Mise à jour de la table Portfolios et Deals pour chaque stratégie

    # Pour la stratégie low_risk
    if df_low_risk is not None:
        update_portfolio(date_str, "low_risk", df_low_risk)
        update_deals(date_str, "low_risk", df_low_risk)
    else:
        print("Portefeuille low_risk non généré.")

    # Pour la stratégie low_turnover, on limite à 2 deals par mois
    if nb_deals < 2:
        if df_low_turnover is not None:
            update_portfolio(date_str, "low_turnover", df_low_turnover)
            update_deals(date_str, "low_turnover", df_low_turnover)
            nb_deals += 1
        else:
            # Si aucun investissement n'est réalisé, on passe new_weight_df=None
            update_deals(date_str, "low_turnover", None)
    else:
        print(f"Pour low_turnover, 2 deals ont déjà été enregistrés en {month_year}, mise à jour ignorée.")

    # Pour la stratégie high_yield_equity_only
    if df_high_yield is not None:
        update_portfolio(date_str, "high_yield_equity_only", df_high_yield)
        update_deals(date_str, "high_yield_equity_only", df_high_yield)
    else:
        print("Portefeuille high_yield_equity_only non généré.")

    print("--------------------------------------------------------\n")

Téléchargement des returns entre 2022-12-26 et 2023-01-02


2025-03-30 22:32:05,113 - ERROR - 
1 Failed download:
2025-03-30 22:32:05,113 - ERROR - ['NG=F']: YFInvalidPeriodError("NG=F: Period 'max' is invalid, must be of the format 1d, 5d, etc.")


Aucun return à insérer (les valeurs étaient NULL, inf ou données manquantes).
--------------------------------------------------------

Téléchargement des returns entre 2023-01-02 et 2023-01-09


2025-03-30 22:32:07,529 - ERROR - 
1 Failed download:
2025-03-30 22:32:07,529 - ERROR - ['NG=F']: YFInvalidPeriodError("NG=F: Period 'max' is invalid, must be of the format 1d, 5d, etc.")


Aucun return à insérer (les valeurs étaient NULL, inf ou données manquantes).
--------------------------------------------------------

Téléchargement des returns entre 2023-01-09 et 2023-01-16


...

2025-03-30 22:38:18,635 - ERROR - 
104 Failed downloads:
2025-03-30 22:38:18,636 - ERROR - ['GOOGL', 'LH', 'UNH', 'PCG', 'HCA', 'PL=F', 'ZW=F', 'BKR', 'ES', 'APA', 'MSFT', 'BZ=F', 'ILMN', 'PSX', 'ZO=F', 'ABT', 'SB=F', 'ADBE', 'META', 'BRK-A', 'NFLX', 'ZB=F', 'CRM', 'HO=F', 'WBA', 'WMT', 'GILD', 'ARE', 'UHS', 'V', 'CCI', 'CMCSA', 'A', 'MRK', 'DVA', 'KR', 'INTC', 'KO', 'TSLA', 'OKE', 'COST', 'EOG', 'NVDA', 'AAPL', 'VRTX', 'SLB', 'VZ', 'SRE', 'NG=F', 'EW', 'XOM', 'PA=F', 'ZF=F', 'D', 'JNJ', 'SI=F', 'DIS', 'KMI', 'BAC', 'VTR', 'HG=F', 'HE=F', 'PEP', 'AEE', 'OXY', 'PG', 'EPD', 'BIIB', 'T', 'ET', 'GF=F', 'ZS=F', 'IQV', 'CC=F', 'RB=F', 'KC=F', 'ZT=F', 'OJ=F', 'PFE', 'DGX', 'HAL', 'HD', 'GC=F', 'JPM', 'DTE', 'MA', 'COP', 'ORCL', 'BMY', 'XEL', 'WELL', 'LLY', 'EIX', 'WMB', 'CMS', 'NI', 'ZN=F', 'ZC=F', 'LE=F', 'CSCO', 'AMZN', 'CT=F', 'ALGN', 'NKE']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Aucune donnée téléchargée depuis Yahoo Finance.
Pour low_turnover, 2 deals ont déjà été enregistrés en 2024-01, mise à jour ignorée.
--------------------------------------------------------

Téléchargement des returns entre 2024-01-15 et 2024-01-22


2025-03-30 22:38:24,406 - ERROR - 
104 Failed downloads:
2025-03-30 22:38:24,408 - ERROR - ['PL=F', 'ZW=F', 'UNH', 'HCA', 'PCG', 'LH', 'GOOGL', 'BKR', 'MSFT', 'ES', 'PSX', 'BZ=F', 'APA', 'ABT', 'ILMN', 'ZO=F', 'BRK-A', 'CRM', 'ADBE', 'SB=F', 'HO=F', 'NFLX', 'ZB=F', 'META', 'UHS', 'ARE', 'WBA', 'GILD', 'WMT', 'CMCSA', 'CCI', 'V', 'A', 'OKE', 'MRK', 'INTC', 'TSLA', 'KO', 'DVA', 'KR', 'AAPL', 'EW', 'SLB', 'EOG', 'NVDA', 'VZ', 'COST', 'VRTX', 'SI=F', 'NG=F', 'SRE', 'D', 'JNJ', 'XOM', 'ZF=F', 'PA=F', 'DIS', 'HG=F', 'PEP', 'EPD', 'BAC', 'VTR', 'KMI', 'HE=F', 'T', 'RB=F', 'BIIB', 'GF=F', 'OXY', 'PG', 'ET', 'AEE', 'CC=F', 'OJ=F', 'ZS=F', 'IQV', 'KC=F', 'DGX', 'HAL', 'ZT=F', 'HD', 'XEL', 'PFE', 'GC=F', 'BMY', 'DTE', 'MA', 'JPM', 'ORCL', 'ZC=F', 'WELL', 'EIX', 'COP', 'NI', 'LLY', 'CMS', 'ZN=F', 'CT=F', 'WMB', 'CSCO', 'AMZN', 'NKE', 'ALGN', 'LE=F']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Aucune donnée téléchargée depuis Yahoo Finance.
Pour low_turnover, 2 deals ont déjà été enregistrés en 2024-01, mise à jour ignorée.
--------------------------------------------------------

Téléchargement des returns entre 2024-01-22 et 2024-01-29


KeyboardInterrupt: 

### Calcul des performances

In [135]:
metrics_lr = PortfolioMetrics("low_risk")
metrics_lt = PortfolioMetrics("low_turnover")
metrics_hy = PortfolioMetrics("high_yield_equity_only")

2025-03-30 18:57:27,455 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 18:57:27,633 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 18:57:27,803 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 18:57:27,973 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 18:57:28,141 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 18:57:28,312 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 18:57:28,479 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 18:57:28,641 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 18:57:28,804 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 18:57:28,965 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 18:57:29,127 - INFO

### Affichage des performances avec streamlit 

In [136]:
#dashboard avec streamlit
!streamlit run dashboard.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.0.26:8501

2025-03-30 19:02:23,499 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 19:02:23,660 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 19:02:23,823 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 19:02:23,983 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 19:02:24,141 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 19:02:24,301 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 19:02:24,458 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 19:02:24,616 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rendements
2025-03-30 19:02:24,773 - INFO - Portefeuille du 2023-01-02: 129 produits, 6 jours de rende